In [1]:
import pandas as pd
import os
import csv
import xlsxwriter

In [2]:
#Primer paso: unificar los archivos csv en una lista de listas
ruta_archivos = r"C:\Users\Usuario\Desktop\Programación\Proyecto Análisis de Datos Entrenos\Datos entrenos nuevos\Sentadilla Frontal"
nombre_archivo = "Datos Sentadilla Frontal.xlsx"

def unificar_archivos(ruta_archivos):
    lista_datos = []
    for archivo in os.listdir(ruta_archivos):
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(ruta_archivos, archivo)
            with open(ruta_archivo, newline='') as archivo_csv:
                lector_csv = csv.reader(archivo_csv)
                for fila in lector_csv:
                    lista_datos.append(fila)
    return lista_datos

lista_datos = unificar_archivos(ruta_archivos)

In [3]:
#Segundo paso: convertir la lista de listas en un dataframe de pandas
def convertir_a_pandas(lista_datos):
    df = pd.DataFrame(lista_datos)
    return df

df = convertir_a_pandas(lista_datos)

In [4]:
#Paso 3: eliminar las filas que no nos interesan
df.drop(df.loc[df[0] == "Atleta"].index, inplace=True)
df.drop(df.loc[df[0] == "Marcado con *: Solo encoder con Bluetooth"].index, inplace=True)
df.drop(df.loc[df[0] == "Email"].index, inplace=True)

In [5]:
#Paso 4: Determinar que nombre quiero para mis columnas y elimnar esas filas
df.columns = df.iloc[0]
df.drop(df.loc[df["Ejercicio"] == "Ejercicio"].index, inplace=True)

In [6]:
#Paso 5: Eliminar las columnas que no me interesan
df.drop("Comentarios", axis=1, inplace=True)

In [7]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [8]:
#Paso 6: Igualar todas las filas en las que no haya datos en la columna ejercicio, fecha, etc.

def rellenar_datos_faltantes(columna):
    valor_del_momento = df.loc[0, columna]
    for indice, valor in df[columna].iteritems():
        if valor == "":
            df.loc[indice, columna] = valor_del_momento
        else:
            valor_del_momento = valor

for columna in df.columns:
    rellenar_datos_faltantes(columna)

In [9]:
#Cambio la fecha a formato date, elimino la hora
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d-%m-%Y %H:%M")
df["Fecha"] = df["Fecha"].dt.date


In [10]:
#Elimino las filas que no tienen datos en la columna serie y en la columna repeticion
df = df.dropna(subset=["nº Serie"])
df = df.dropna(subset=["nº Rep."])


In [11]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [12]:
#Transformo los datos de las columnas que van a contener datos numericos
columnas_no_numericas = ["Ejercicio", "Fecha", "Tipo*"]
for columna in df.columns:
    if columna not in columnas_no_numericas:
        df[columna] = df[columna].str.replace(",", ".")
        df[columna] = pd.to_numeric(df[columna])


In [13]:
#Convierto columna fecha a formato date
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")

In [14]:
df

3,Ejercicio,Fecha,nº Serie,Peso (kg),nº Rep.,Tipo*,VMP (m/s),Velocidad máxima (m/s),Potencia Media (W),ROM (cm),1RM (kg)
0,SENTADILLA FRONTAL,2023-01-30,1,20.0,1,C,1.23,2.10,241.33,100.90,44.99
1,SENTADILLA FRONTAL,2023-01-30,2,50.0,1,C,0.86,1.51,421.83,86.51,73.95
2,SENTADILLA FRONTAL,2023-01-30,2,50.0,2,C,0.90,1.67,441.45,89.83,76.79
3,SENTADILLA FRONTAL,2023-01-30,2,50.0,3,C,0.89,1.62,436.54,92.30,76.06
4,SENTADILLA FRONTAL,2023-01-30,2,50.0,4,C,0.90,1.58,441.45,93.71,76.79
...,...,...,...,...,...,...,...,...,...,...,...
100,SENTADILLA FRONTAL,2023-03-06,2,85.0,2,C,0.41,1.02,341.88,81.04,88.74
101,SENTADILLA FRONTAL,2023-03-06,3,80.0,1,C,0.57,1.26,447.34,82.65,93.28
102,SENTADILLA FRONTAL,2023-03-06,3,80.0,2,C,0.50,1.08,392.40,83.30,88.74
103,SENTADILLA FRONTAL,2023-03-06,4,85.0,1,C,0.59,1.30,491.97,80.84,100.57


In [45]:
#Añado nuevas columnas con datos que me interesan
df_fatiga = df.groupby(["Ejercicio", "Fecha", "nº Serie", "Peso (kg)"]).agg({"VMP (m/s)" : ["min", "max"]})
df_fatiga["Fatiga"] = df_fatiga["VMP (m/s)"]["max"] - df_fatiga["VMP (m/s)"]["min"]
df_fatiga["Porcentaje de fatiga"] = df_fatiga["Fatiga"] / df_fatiga["VMP (m/s)"]["max"] * 100
df_fatiga.reset_index(inplace=True)
df_fatiga

3            Ejercicio      Fecha nº Serie Peso (kg) VMP (m/s)       Fatiga  \
                                                           min   max          
0   SENTADILLA FRONTAL 2023-01-30        1      20.0      1.23  1.23   0.00   
1   SENTADILLA FRONTAL 2023-01-30        2      50.0      0.79  0.90   0.11   
2   SENTADILLA FRONTAL 2023-01-30        3      50.0      0.77  0.90   0.13   
3   SENTADILLA FRONTAL 2023-01-30        4      50.0      0.79  0.91   0.12   
4   SENTADILLA FRONTAL 2023-02-07        1      20.0      0.61  1.27   0.66   
5   SENTADILLA FRONTAL 2023-02-07        2      50.0      0.89  0.89   0.00   
6   SENTADILLA FRONTAL 2023-02-07        3      60.0      0.78  0.78   0.00   
7   SENTADILLA FRONTAL 2023-02-07        4      65.0      0.66  0.74   0.08   
8   SENTADILLA FRONTAL 2023-02-07        5      70.0      0.58  0.68   0.10   
9   SENTADILLA FRONTAL 2023-02-07        6      70.0      0.60  0.67   0.07   
10  SENTADILLA FRONTAL 2023-02-09        1      60.0      0.73  0.82   0.09   
11  SENTADILLA FRONTAL 2023-02-09        2      60.0      0.69  0.83   0.14   
12  SENTADILLA FRONTAL 2023-02-09        3      60.0      0.52  0.80   0.28   
13  SENTADILLA FRONTAL 2023-02-14        1      75.0      0.50  0.61   0.11   
14  SENTADILLA FRONTAL 2023-02-14        2      75.0      0.50  0.60   0.10   
15  SENTADILLA FRONTAL 2023-02-14        3      70.0      0.47  0.66   0.19   
16  SENTADILLA FRONTAL 2023-02-19        1      65.0      0.58  0.62   0.04   
17  SENTADILLA FRONTAL 2023-02-19        2      60.0      0.62  0.75   0.13   
18  SENTADILLA FRONTAL 2023-02-19        3      60.0      0.64  0.78   0.14   
19  SENTADILLA FRONTAL 2023-02-19        4      60.0      0.61  0.76   0.15   
20  SENTADILLA FRONTAL 2023-02-21        1      60.0      0.57  0.71   0.14   
21  SENTADILLA FRONTAL 2023-02-21        2      70.0      0.60  0.70   0.10   
22  SENTADILLA FRONTAL 2023-02-21        3      75.0      0.46  0.59   0.13   
23  SENTADILLA FRONTAL 2023-02-21        4      75.0      0.50  0.59   0.09   
24  SENTADILLA FRONTAL 2023-02-23        1      65.0      0.57  0.70   0.13   
25  SENTADILLA FRONTAL 2023-02-23        2      65.0      0.63  0.80   0.17   
26  SENTADILLA FRONTAL 2023-02-23        3      65.0      0.74  0.88   0.14   
27  SENTADILLA FRONTAL 2023-02-27        1      70.0      0.53  0.63   0.10   
28  SENTADILLA FRONTAL 2023-02-27        2      75.0      0.56  0.64   0.08   
29  SENTADILLA FRONTAL 2023-02-27        3      80.0      0.47  0.53   0.06   
30  SENTADILLA FRONTAL 2023-02-27        4      80.0      0.52  0.63   0.11   
31  SENTADILLA FRONTAL 2023-03-01        1      70.0      0.52  0.62   0.10   
32  SENTADILLA FRONTAL 2023-03-01        2      65.0      0.53  0.63   0.10   
33  SENTADILLA FRONTAL 2023-03-01        3      65.0      0.70  0.76   0.06   
34  SENTADILLA FRONTAL 2023-03-06        1      60.0      0.68  0.78   0.10   
35  SENTADILLA FRONTAL 2023-03-06        2      85.0      0.41  0.51   0.10   
36  SENTADILLA FRONTAL 2023-03-06        3      80.0      0.50  0.57   0.07   
37  SENTADILLA FRONTAL 2023-03-06        4      85.0      0.45  0.59   0.14   

3  Porcentaje de fatiga  
                         
0              0.000000  
1             12.222222  
2             14.444444  
3             13.186813  
4             51.968504  
5              0.000000  
6              0.000000  
7             10.810811  
8             14.705882  
9             10.447761  
10            10.975610  
11            16.867470  
12            35.000000  
13            18.032787  
14            16.666667  
15            28.787879  
16             6.451613  
17            17.333333  
18            17.948718  
19            19.736842  
20            19.718310  
21            14.285714  
22            22.033898  
23            15.254237  
24            18.571429  
25            21.250000  
26            15.909091  
27            15.873016  
28            12.500000  
29            11.320755  
30     

In [49]:
#Creo la función para eliminar las filas que sean menores a las filas con máximo peso
def eliminar_filas(df):
    max_peso = df['Peso (kg)'].max()
    return df[df['Peso (kg)'] == max_peso]

df_fatiga = df_fatiga.groupby(["Ejercicio", "Fecha"]).apply(eliminar_filas)
df_fatiga.reset_index(drop=True, inplace=True)
df_fatiga

3            Ejercicio      Fecha nº Serie Peso (kg) VMP (m/s)       Fatiga  \
                                                           min   max          
0   SENTADILLA FRONTAL 2023-01-30        2      50.0      0.79  0.90   0.11   
1   SENTADILLA FRONTAL 2023-01-30        3      50.0      0.77  0.90   0.13   
2   SENTADILLA FRONTAL 2023-01-30        4      50.0      0.79  0.91   0.12   
3   SENTADILLA FRONTAL 2023-02-07        5      70.0      0.58  0.68   0.10   
4   SENTADILLA FRONTAL 2023-02-07        6      70.0      0.60  0.67   0.07   
5   SENTADILLA FRONTAL 2023-02-09        1      60.0      0.73  0.82   0.09   
6   SENTADILLA FRONTAL 2023-02-09        2      60.0      0.69  0.83   0.14   
7   SENTADILLA FRONTAL 2023-02-09        3      60.0      0.52  0.80   0.28   
8   SENTADILLA FRONTAL 2023-02-14        1      75.0      0.50  0.61   0.11   
9   SENTADILLA FRONTAL 2023-02-14        2      75.0      0.50  0.60   0.10   
10  SENTADILLA FRONTAL 2023-02-19        1      65.0      0.58  0.62   0.04   
11  SENTADILLA FRONTAL 2023-02-21        3      75.0      0.46  0.59   0.13   
12  SENTADILLA FRONTAL 2023-02-21        4      75.0      0.50  0.59   0.09   
13  SENTADILLA FRONTAL 2023-02-23        1      65.0      0.57  0.70   0.13   
14  SENTADILLA FRONTAL 2023-02-23        2      65.0      0.63  0.80   0.17   
15  SENTADILLA FRONTAL 2023-02-23        3      65.0      0.74  0.88   0.14   
16  SENTADILLA FRONTAL 2023-02-27        3      80.0      0.47  0.53   0.06   
17  SENTADILLA FRONTAL 2023-02-27        4      80.0      0.52  0.63   0.11   
18  SENTADILLA FRONTAL 2023-03-01        1      70.0      0.52  0.62   0.10   
19  SENTADILLA FRONTAL 2023-03-06        2      85.0      0.41  0.51   0.10   
20  SENTADILLA FRONTAL 2023-03-06        4      85.0      0.45  0.59   0.14   

3  Porcentaje de fatiga  
                         
0             12.222222  
1             14.444444  
2             13.186813  
3             14.705882  
4             10.447761  
5             10.975610  
6             16.867470  
7             35.000000  
8             18.032787  
9             16.666667  
10             6.451613  
11            22.033898  
12            15.254237  
13            18.571429  
14            21.250000  
15            15.909091  
16            11.320755  
17            17.460317  
18            16.129032  
19            19.607843  
20            23.728814

In [53]:
#Obtengo la media de los porcentajes de fatiga por fecha
df_fatiga.columns = ["Ejercicio", "Fecha", "nº Serie", "Peso (kg)", "VMP (m/s) min", "VMP (m/s) max", "Fatiga", "Porcentaje de fatiga"]
df_fatiga_media = df_fatiga.groupby(["Fecha"]).agg({"Porcentaje de fatiga" : "mean"})
df_fatiga_media.reset_index(inplace=True)
df_fatiga_media


,Fecha,Porcentaje de fatiga
0,2023-01-30,13.284493
1,2023-02-07,12.576822
2,2023-02-09,20.947693
3,2023-02-14,17.349727
4,2023-02-19,6.451613
5,2023-02-21,18.644068
6,2023-02-23,18.576840
7,2023-02-27,14.390536
8,2023-03-01,16.129032
9,2023-03-06,21.668328


In [56]:
#Convierto dataframe a excel
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
df_fatiga_media.to_excel(writer, sheet_name='Sheet2')
writer.save()
